In [1]:
# from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import LlamaForCausalLM, LlamaTokenizer
from huggingface_hub import login
import os

[2023-08-02 20:18:39,604] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [2]:
model_name = "meta-llama/Llama-2-7b-hf"

In [3]:
login()

In [4]:
# tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

In [6]:
model = LlamaForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
problem = "A $90^\\circ$ rotation around the origin in the counter-clockwise direction is applied to $7 + 2i.$  What is the resulting complex number?"
steps = [
    "I know that a $90^\\circ$ rotation around the origin in the complex plane can be achieved by multiplying the complex number by $i$, since $i$ has a magnitude of 1 and an argument of $90^\\circ$.",
    "So, I can write the rotation as $(7 + 2i)i = 7i + 2i^2.$",
    "To simplify this expression, I recall that $i^2 = -1$, so I can substitute that and get $7i - 2.$",
    "This is the resulting complex number after the rotation.\n\n# Answer\n\n7i - 2",
]

In [ ]:
model.generate